# Milestone P4 - A comparative approach on chilling effect, from Wikipedia to Google Trend

## Course CS-401 - Applied Data Analysis

### Instructor : [Robert West](https://dlab.epfl.ch/people/west/)

### Author :  
- **Chraibi Ghali**  
    SCIPER: 262251
- **Jesslen Artur**  
    SCIPER: 270642
- **Michels Luc**  
    SCIPER: ????????????  

#### Due date: 18 Dec 2020

---
### Context

Lorem ipsum

### Data

Lorem ipsum

### Imports

In [ ]:
#!pip install pickle5
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd
import numpy as np
import pickle5 as pickle

from os import path

---
### Replicate the experiment of the paper

#### Load data

In [ ]:
data_path = 'data/'

if path.exists(data_path):
    with open(data_path+"terrorism.pkl", "rb") as ter:
        terrorism_df = pickle.load(ter)
    with open(data_path+"domestic.pkl", "rb") as dom:
        domestic_df = pickle.load(dom)
    with open(data_path+"top_30_terrorism.pkl", "rb") as top_ter:
        top_30_terrorism_df = pickle.load(top_ter)
else:
    raise NotADirectoryError

In [ ]:
terrorism_df = terrorism_df[['date', 'max_ratio', 'topic_name']]
print(terrorism_df.shape)
terrorism_df.head(5)

#### Analyse data

In [ ]:
# Define a constant for the date of revelation
revelation_date = pd.Timestamp('2013-06-15')

In [ ]:
# Organise the data in a time series with the ratio of total views per week
terrorism_ts = terrorism_df.groupby(by='date').sum()
terrorism_ts = terrorism_ts.reset_index().rename(columns={'index': 'date'})

# Separate the data in two to perform an ITS analysis
terrorism_pre_june_ts = terrorism_ts[terrorism_ts['date'] < revelation_date]
terrorism_post_june_ts = terrorism_ts[terrorism_ts['date'] > revelation_date]

Step 1

In [ ]:
print(np.mean(terrorism_pre_june_ts['max_ratio']))
print(np.mean(terrorism_post_june_ts['max_ratio']))
print(terrorism_post_june_ts.shape)
terrorism_pre_june_ts

In [ ]:
mean_pre_june = np.mean(terrorism_pre_june_ts['max_ratio'])
mean_post_june = np.mean(terrorism_post_june_ts['max_ratio'])
plt.bar([0, 1], [mean_pre_june, mean_post_june], color='gray', width=0.4)
plt.xticks([0, 1], ['Pre June, 2013', 'Post June, 2013'])
plt.ylabel("Average interest")
plt.xlim(-0.5, 1.5)
plt.ylim(0, 200)
plt.text(-0.1, mean_pre_june + 5, f"{mean_pre_june:.2f}")
plt.text(0.9, mean_post_june + 5, f"{mean_post_june:.2f}")
plt.show()

Step 2

In [ ]:
### TODO Modify description of the plot

# Visualisation of full time series
fig, ax = plt.subplots(figsize=(10, 6))

ax.set_title('Figure 2. Pre and Post June 2013 Topic Ratio Trends \n\n', fontsize=14)
ax.set_xlabel('\nTime (?Weeks?)', fontsize=11)
ax.set_ylabel('Total views (All ?48? Google Articles)\n', fontsize=11)

ax.scatter(terrorism_ts.date, terrorism_ts['max_ratio'], label='Total Articles Views (By Month)', color='black')

# Draw the regressions
slope_pre_june, intercept_pre_june, _, _, _ = stats.linregress(terrorism_pre_june_ts.index, 
                                                               terrorism_pre_june_ts['max_ratio'])
slope_post_june, intercept_post_june, _, _, _ = stats.linregress(terrorism_post_june_ts.index,
                                                                 terrorism_post_june_ts['max_ratio'])
x1 = terrorism_pre_june_ts.index
ax.plot(terrorism_pre_june_ts.date, intercept_pre_june + slope_pre_june*x1, color='black', linewidth='3', label='Trend Pre-June 2013')
x2 = terrorism_post_june_ts.index
ax.plot(terrorism_post_june_ts.date, intercept_post_june + slope_post_june*x2, color='gray', linewidth='3', label='Trend Post-June 2013')

# Emphasise when the revelations occured 
ax.axvline(x=revelation_date, ymin=0, ymax=280, color='black', linewidth='3')

x_text_offset = revelation_date - pd.Timedelta(50, 'd')
y_text_offset = 305
ax.text(x_text_offset, y_text_offset, 'Mid June 2013', fontsize=10)

ax.set_ylim(0, 300)

ax.legend(loc='lower left')
plt.show()

Step 3 - Compare